In [ ]:
# # This notebooks looks at primary mirror motion during images as a function of elevation

In [ ]:
import asyncio 
import matplotlib

import numpy as np
import pandas as pd

from matplotlib import pylab as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient, rendezvous_dataframes

In [ ]:
%matplotlib inline

In [ ]:
#efd_client = EfdClient('summit_efd')
efd_client = EfdClient('ldf_stable_efd') 

Query for all the `endReadout` events on the timespan of the night, elevation, and pmd data

In [ ]:
# base = await efd_client.select_time_series("lsst.sal.ATCamera.logevent_endReadout", 
#                                            ["imageName", "requestedExposureTime", "additionalKeys", "additionalValues"], t1, t2)

In [ ]:
# Or can base it on script messages - but these can fail since things don't always go in it

In [ ]:
# night of 2021-07-08 - This set had the optics in a weird state, not enough air underneath
# t1 = Time("2021-07-09T00:28:20", format='isot', scale='tai')
# t2 = Time("2021-07-09T00:57:26", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-08 - TEst run
t1 = Time("2021-07-14T20:24:30", format='isot', scale='tai')
t2 = Time("2021-07-14T20:32:07", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run - valves off, ataos corrections disabled
t1 = Time("2021-07-15T18:28:20", format='isot', scale='tai')
t2 = Time("2021-07-15T18:36:23", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 2 - valves off, ataos corrections disabled
t1 = Time("2021-07-15T18:41:28", format='isot', scale='tai')
t2 = Time("2021-07-15T18:48:17", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 3 - valves off, ataos corrections disabled
t1 = Time("2021-07-15T18:49:00", format='isot', scale='tai')
t2 = Time("2021-07-15T18:56:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 1 - valves open, ataos corrections disabled
t1 = Time("2021-07-15T19:02:00", format='isot', scale='tai')
t2 = Time("2021-07-15T19:10:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 2 - valves open, ataos corrections disabled
t1 = Time("2021-07-15T19:14:00", format='isot', scale='tai')
t2 = Time("2021-07-15T19:22:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 1 - valves open, ataos corrections enabled, scale_factor=1
t1 = Time("2021-07-15T19:45:10", format='isot', scale='tai')
t2 = Time("2021-07-15T19:52:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 2 - valves open, ataos corrections enabled, scale_factor=1
t1 = Time("2021-07-15T20:09:57", format='isot', scale='tai')
t2 = Time("2021-07-15T20:17:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 2c - valves open, ataos corrections enabled, offset by -10%, adjusted elevation range to stay out of extrapolation area
t1 = Time("2021-07-15T21:01:10", format='isot', scale='tai')
t2 = Time("2021-07-15T21:07:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# day of 2021-07-15 - Test run 2c - valves open, ataos corrections enabled, offset by -5%, adjusted elevation range to stay out of extrapolation area
t1 = Time("2021-07-15T21:09:10", format='isot', scale='tai')
t2 = Time("2021-07-15T21:18:00", format='isot', scale='tai')#+TimeDelta(8.*24.*60*60., format='sec', scale='tai')

In [ ]:
# # night of 2021-07-08 - Set after having mitutotyo 5 adjusted
# #t1 = Time('1625805905.409408', format='unix', scale='tai')-TimeDelta(37, format='sec', scale='tai')
# t1 = Time('1625805905.409408', format='unix', scale='tai')-TimeDelta(37, format='sec')
# t2 = t1+TimeDelta(600, format='sec')
# print(t1.isot)
# print(t2.isot)

In [ ]:
base0 = await efd_client.select_time_series("lsst.sal.Script.logevent_logMessage", 
                                           ["message","level"], t1, t2)
base0

In [ ]:
# ind= (np.where(base0.message.str.find('az,el,pos0,pos1,pos2') != -1))
# print(ind)
# base0.iloc[ind]

In [ ]:
# base = base0.iloc[ind[0][:]]

In [ ]:
base=base0

In [ ]:
el = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", ["elevationCalculatedAngle1", ], t1, t2)
# mount reporting incorrect timestamp
el.index=el.index+pd.tseries.offsets.DateOffset(seconds=-37)

In [ ]:
mount_Nasmyth_Encoders = await efd_client.select_packed_time_series("lsst.sal.ATMCS.mount_Nasmyth_Encoders",
                                                                    ["nasmyth2CalculatedAngle"], t1 , t2)
mount_Nasmyth_Encoders.index=mount_Nasmyth_Encoders.index+pd.tseries.offsets.DateOffset(seconds=-37)

In [ ]:
pmd = await efd_client.select_time_series("lsst.sal.PMD.position", ["position0", "position1", "position2", "position3", "position4"], t1, t2)

In [ ]:
hexapod_vals = await efd_client.select_time_series("lsst.sal.ATHexapod.positionStatus", 
                                       ["reportedPosition0", "reportedPosition1", "reportedPosition2",
                                       "reportedPosition3", "reportedPosition4", "reportedPosition5"], t1 , t2)

In [ ]:
hex_elevation=rendezvous_dataframes(hexapod_vals,el)

In [ ]:
tmp = rendezvous_dataframes(base,el)

In [ ]:
# tmp2a = rendezvous_dataframes(tmp, m1_x_pos)
# tmp2b = rendezvous_dataframes(tmp2a, m1_y_pos)
# tmp2c = rendezvous_dataframes(tmp2b, m1_z_piston)
# tmp2d = rendezvous_dataframes(tmp2c, m1_tip_y)
# tmp2e = rendezvous_dataframes(tmp2d, m1_tilt_x)
tmp2= rendezvous_dataframes(tmp, pmd)
tmp2a= rendezvous_dataframes(tmp2, pmd)
# tmp3 = rendezvous_dataframes(tmp2,end_readout)
tmp4 = rendezvous_dataframes(tmp2,hexapod_vals)
raw_data = tmp4

In [ ]:
# position0_offset = np.mean(raw_data.position0)
# position1_offset = np.mean(raw_data.position1)
# position2_offset = np.mean(raw_data.position2)
# position3_offset = np.mean(raw_data.position3)
# position4_offset = np.mean(raw_data.position4)

position0_offset = (raw_data.position0[0])
position1_offset = (raw_data.position1[0])
position2_offset = (raw_data.position2[0])
position3_offset = (raw_data.position3[0])
position4_offset = (raw_data.position4[0])

In [ ]:
raw_data

In [ ]:
from scipy import linalg
arr_len = len(raw_data.position0)
coeff_arr = np.zeros((arr_len,3))
theta_arr = np.zeros((arr_len))
phi_arr = np.zeros((arr_len))
piston_arr = np.zeros((arr_len))
for i in np.arange(arr_len):
#     data=np.c_[(raw_data.position2[i]-position2_offset), (raw_data.position2[i]-position2_offset), (raw_data.position2[i]-position2_offset)]
    # X, Y, Z
    set2=np.array((  41.0, 468.0, (raw_data.position2[i]-position2_offset)))
    set3=np.array(( 384.0,-269.0, (raw_data.position3[i]-position3_offset)))
    set4=np.array((-425.0,-198.0, (raw_data.position4[i]-position4_offset)))
#     data=np.c_[set2, set3, set4]    
#     print(f'{i} is {data}')
    # regular grid covering the domain of the data
#     mn = np.min(data, axis=0)
#     mx = np.max(data, axis=0)
#     X,Y = np.meshgrid(np.linspace(mn[0], mx[0], 20), np.linspace(mn[1], mx[1], 20))
#     XX = X.flatten()
#     YY = Y.flatten()

#     # best-fit linear plane (1st-order)
#     A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
#     C,_,_,_ = linalg.lstsq(A, data[:,2])    # coefficients

#     # evaluate it on grid
#     Z = C[0]*X + C[1]*Y + C[2]
#     coeff_arr[i,:] = C[0], C[1], C[2]
    #print(f'C0, C1, and C2 is {C[0]}, {C[1]}, {C[2]}')
    
    # Piston at x=0, y=0
#     piston_arr[i] = C[2]
    #
    
    # Vector PQ crossed with Vector PR
    normal = np.cross(set3-set2,set4-set2) # gives a,b,c
    #print(f'normal is {normal}')
#     theta_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[0])) * 206265 # arcsec
#     phi_arr[i] = (np.pi/2 + np.arctan2(normal[2],normal[1])) * 206265   # arcsec
#     piston_arr[i] = normal[2]
    
#     => a * (x - x0) + b * (y - y0) + c * (z - z0) = 0.
# => a * x - a * x0 + b * y - b * y0 + c * z - c * z0 = 0.
# => a * x + b * y + c * z + (- a * x0 - b * y0 - c * z0) = 0. # D is the last terms
    D= -normal[0]*set2[0] - normal[1]*set2[1] - normal[2]*set2[2]  # Constant in plane equation
    # equation 
    
    phi_from_normal = (np.pi/2+np.arctan2(normal[2], normal[1])) * 206265
    theta_from_normal = (np.pi/2+np.arctan2(normal[2], normal[0])) * 206265
    # find z at the origin to represent piston
    Z_origin = -D/normal[2]
    
    # Measure rotation about the Y-axis (perpendicular to elevation)
    # So this is TIP and should result in motion in azimuth
    # get slope by looking at Y=0, X=400
    x_pt=400; y_pt=0
    #Z_at_x_pt= C[0]*x_pt + C[1]*0.0 + C[2]
    Z_at_x_pt= (-D - normal[0]*x_pt - normal[1]*0.0)/normal[2]
    theta = np.arctan2(Z_at_x_pt-Z_origin, x_pt) * 206265 # arcsec
        
    # Measure rotation about the X-axis (aligned to elevation)
    # this is TILT and should result in motion in elevation
    # get slope by looking at Y=0, X=400    
    x_pt=0; y_pt=400
    Z_at_y_pt= (-D - normal[0]*x_pt - normal[1]*y_pt)/normal[2]
    phi = np.arctan2(Z_at_y_pt-Z_origin, y_pt) * 206265 # arcsec

    theta_arr[i] = theta # arcsec
    phi_arr[i] = phi    # arcsec
    piston_arr[i] = Z_origin
    
    print(f'theta_from_normal is {theta_from_normal:0.2f}, phi_from_normal is {phi_from_normal:0.2f}')
    print(f'theta_arr is {theta_arr[i]:0.2f} [arcsec], phi_arr is {phi_arr[i]:0.2f} [arcsec]')
#     if i == 1:
#         break

In [ ]:
import copy
calc_data=copy.copy(raw_data)

In [ ]:
calc_data['m1_tip']=theta_arr
calc_data['m1_tilt']=phi_arr
calc_data['m1_piston']=piston_arr

calc_data['m1_y_pos']=(raw_data.position0-position0_offset)*np.cos(10*np.pi/180)
calc_data['m1_x_pos']=(raw_data.position1-position1_offset)*np.cos(10*np.pi/180)

In [ ]:
fig_height=5
fig_width=10

In [ ]:
nwide=3; nhigh=3
fig, (row1,row2,row3) = plt.subplots(nhigh, nwide, figsize=(nwide+fig_width, nhigh*fig_height))
fig.suptitle('Rows are')
xvals = (calc_data.m1_x_pos) # um
yvals = calc_data.elevationCalculatedAngle1

row1[0].plot(xvals, yvals, 'o-')
row1[0].set_ylabel('Elevation [deg]')
row1[0].set_xlabel('M1 X-position [mm]')

xvals=calc_data.reportedPosition1
row1[1].plot(xvals,yvals, '.-')
row1[1].set_xlabel('Hexapod Y-position [mm]')


xvals = (calc_data.m1_tip) # um
row1[2].plot(xvals, yvals, 'o-')
row1[2].set_ylabel('Elevation [deg]')
row1[2].set_xlabel('M1 tip [arcsec]')


xvals = (calc_data.m1_y_pos) # mm
row2[0].plot(xvals, yvals, 'o-')
row2[0].set_ylabel('Elevation [deg]')
row2[0].set_xlabel('M1 X-position [mm]')

xvals=calc_data.reportedPosition0
row2[1].plot(xvals,yvals, '.-')
row2[1].set_xlabel('Hexapod X-position [mm]')

xvals = (calc_data.m1_tilt) # um
row2[2].plot(xvals, yvals, 'o-')
row2[2].set_ylabel('Elevation [deg]')
row2[2].set_xlabel('M1 tilt [arcsec]')

xvals = (calc_data.m1_piston) # mm
row3[0].plot(xvals, yvals, 'o-')
row3[0].set_ylabel('Elevation [deg]')
row3[0].set_xlabel('M1 Z-position [mm]')

xvals=calc_data.reportedPosition2
row3[1].plot(xvals,yvals, '.-')
row3[1].set_xlabel('Hexapod Z-position [mm]')


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    hex_elevation

In [ ]:
centroid_arr[i,:]=(data.brightestObjCentroid)

In [ ]:
data.brightestObjCentroid

In [ ]:
centroid_arr[0,:]

In [ ]:
name